In [ ]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
data = pd.read_csv("/content/description.txt")
print(data.head())


                              Train data:
0  ID ::: TITLE ::: GENRE ::: DESCRIPTION
1  ID ::: TITLE ::: GENRE ::: DESCRIPTION
2  ID ::: TITLE ::: GENRE ::: DESCRIPTION
3  ID ::: TITLE ::: GENRE ::: DESCRIPTION
4                              Test data:


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text


In [ ]:
print(data.columns)


Index(['Train data:'], dtype='object')


In [ ]:
print(list(data.columns))

['Train data:']


In [ ]:
print(data.dtypes)


Train data:    object
dtype: object


In [ ]:
text_columns = data.select_dtypes(include='object').columns
print("Text columns:", text_columns)


Text columns: Index(['Train data:'], dtype='object')


In [ ]:
data.head(10)


,Train data:
0,ID ::: TITLE ::: GENRE ::: DESCRIPTION
1,ID ::: TITLE ::: GENRE ::: DESCRIPTION
2,ID ::: TITLE ::: GENRE ::: DESCRIPTION
3,ID ::: TITLE ::: GENRE ::: DESCRIPTION
4,Test data:
5,ID ::: TITLE ::: DESCRIPTION
6,ID ::: TITLE ::: DESCRIPTION
7,ID ::: TITLE ::: DESCRIPTION
8,ID ::: TITLE ::: DESCRIPTION
9,Source:


In [ ]:
data.iloc[0]


,0
Train data:,ID ::: TITLE ::: GENRE ::: DESCRIPTION


In [ ]:
data['Train data:'].head(10)


,Train data:
0,ID ::: TITLE ::: GENRE ::: DESCRIPTION
1,ID ::: TITLE ::: GENRE ::: DESCRIPTION
2,ID ::: TITLE ::: GENRE ::: DESCRIPTION
3,ID ::: TITLE ::: GENRE ::: DESCRIPTION
4,Test data:
5,ID ::: TITLE ::: DESCRIPTION
6,ID ::: TITLE ::: DESCRIPTION
7,ID ::: TITLE ::: DESCRIPTION
8,ID ::: TITLE ::: DESCRIPTION
9,Source:


In [ ]:
data['Train data:'].iloc[0]

'ID ::: TITLE ::: GENRE ::: DESCRIPTION'

In [ ]:
data[['genre', 'plot']] = data['Train data:'].str.split(':', n=1, expand=True)


In [ ]:
split_data = data['Train data:'].str.extract(r'^\s*([^:|,]+)\s*[:|,]\s*(.+)$')
split_data.columns = ['genre', 'plot']

data = pd.concat([data, split_data], axis=1)
data.drop(columns=['Train data:'], inplace=True)


In [ ]:
print(data.head())
print(data.columns)


       genre                                plot genre  \
0        ID   :: TITLE ::: GENRE ::: DESCRIPTION   ID    
1        ID   :: TITLE ::: GENRE ::: DESCRIPTION   ID    
2        ID   :: TITLE ::: GENRE ::: DESCRIPTION   ID    
3        ID   :: TITLE ::: GENRE ::: DESCRIPTION   ID    
4  Test data                                       NaN   

                                 plot  
0  :: TITLE ::: GENRE ::: DESCRIPTION  
1  :: TITLE ::: GENRE ::: DESCRIPTION  
2  :: TITLE ::: GENRE ::: DESCRIPTION  
3  :: TITLE ::: GENRE ::: DESCRIPTION  
4                                 NaN  
Index(['genre', 'plot', 'genre', 'plot'], dtype='object')


In [ ]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

data['plot'] = data['plot'].apply(clean_text)


/tmp/ipython-input-4113872515.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['plot'] = data['plot'].apply(clean_text)


In [ ]:
data['plot'] = data['plot'].apply(clean_text)


/tmp/ipython-input-1061363166.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data['plot'] = data['plot'].apply(clean_text)


In [ ]:
X = data['plot']
y = data['genre']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
print(type(y_train))
print(y_train.shape)
print(y_train.head())


<class 'pandas.core.frame.DataFrame'>
(8, 2)
        genre genre
10        ftp   ftp
2         ID    ID 
1         ID    ID 
8         ID    ID 
4   Test data   NaN


In [ ]:
data = data.loc[:, ~data.columns.duplicated()]


In [ ]:
print(data.columns)



Index(['genre', 'plot'], dtype='object')


In [ ]:
genre = "Test data"


In [ ]:
data = data[data['genre'].notna()]
data = data[data['genre'] != 'Test data']


In [ ]:
y = data['genre']


In [ ]:
from sklearn.model_selection import train_test_split

X = data['plot']
y = data['genre']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
print(type(y_train))
print(y_train.shape)


<class 'pandas.core.series.Series'>
(8,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

y_pred = nb.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.5
              precision    recall  f1-score   support

         ID        0.50      1.00      0.67         1
      Source       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)

y_pred_lr = lr.predict(X_test_tfidf)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Logistic Regression Accuracy: 0.5
              precision    recall  f1-score   support

         ID        0.50      1.00      0.67         1
      Source       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)

y_pred_svm = svm.predict(X_test_tfidf)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.5
              precision    recall  f1-score   support

         ID        0.50      1.00      0.67         1
      Source       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
